### Clean all data together so that the categorical conversion is applicable.!

In [37]:
import pandas as pd
import json
from pandas import json_normalize

df=pd.read_csv(r'./parsed_data/all_cars_parsed_data.csv',header=[0])
df.head(2)

C:\Users\VINOTH\AppData\Local\Temp\ipykernel_17084\1282186920.py:5: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r'./parsed_data/all_cars_parsed_data.csv',header=[0])


,City,it,ft,bt,km,transmission,ownerNo,owner,oem,model,...,Front Brake Type,Rear Brake Type,Top Speed,Acceleration,Tyre Type,No Door Numbers,Cargo Volumn,Wheel Size,Alloy Wheel Size,Ground Clearance Unladen
0,Bangalore,0,Petrol,Hatchback,"1,20,000",Manual,3,3rd Owner,Maruti,Maruti Celerio,...,Ventilated Disc,Drum,150 Kmph,15.05 Seconds,"Tubeless, Radial",5.0,235-litres,NaN,NaN,NaN
1,Bangalore,0,Petrol,SUV,"32,706",Manual,2,2nd Owner,Ford,Ford Ecosport,...,Ventilated Disc,Drum,NaN,NaN,"Tubeless,Radial",4.0,352-litres,16,16,NaN


- Keep int64, float64, and bool as-is
- Attempt to convert object columns to numeric — if successful, assign the result
- Attempt boolean conversion for text-based columns like 'yes'/'no', 'true'/'false'
- Leave everything else as object

In [38]:
# Attempt to convert each column
for col in df.columns:
    # Skip already numeric or boolean
    if pd.api.types.is_numeric_dtype(df[col]) or pd.api.types.is_bool_dtype(df[col]):
        continue
    
    # Try converting to numeric (int/float)
    converted = pd.to_numeric(df[col], errors='coerce')
    
    # If it successfully converts (not all NaNs), use it
    if not converted.isna().all():
        df[col] = converted
        continue
    
    # Try boolean conversion for strings like 'yes', 'no', 'true', 'false'
    unique_vals = df[col].dropna().str.lower().unique()
    if set(unique_vals).issubset({'yes', 'no', 'true', 'false', '0', '1'}):
        df[col] = df[col].str.lower().map({'yes': True, 'no': False, 'true': True, 'false': False, '1': True, '0': False})


### check col dtype

In [39]:
# Group similar types together
def get_count_dtypes(df):
    dtype_counts = {
        'int': 0,
        'float': 0,
        'bool': 0,
        'object': 0,
        'datetime': 0,
        'other': 0
    }
    
    for dtype in df.dtypes:
        if pd.api.types.is_integer_dtype(dtype):
            dtype_counts['int'] += 1
        elif pd.api.types.is_float_dtype(dtype):
            dtype_counts['float'] += 1
        elif pd.api.types.is_bool_dtype(dtype):
            dtype_counts['bool'] += 1
        elif pd.api.types.is_object_dtype(dtype):
            dtype_counts['object'] += 1
        elif pd.api.types.is_datetime64_any_dtype(dtype):
            dtype_counts['datetime'] += 1
        else:
            dtype_counts['other'] += 1
    
    # Display the count
    print(pd.Series(dtype_counts))


In [40]:
def convert_object_to_int_float(df):
    # Attempt to convert each column
    for col in df.columns:
        # Skip already numeric or boolean
        if pd.api.types.is_numeric_dtype(df[col]) or pd.api.types.is_bool_dtype(df[col]):
            continue
        
        # Try converting to numeric (int/float)
        converted = pd.to_numeric(df[col], errors='coerce')
        
        # If it successfully converts (not all NaNs), use it
        if not converted.isna().all():
            df[col] = converted
            continue
        
        # Try boolean conversion for strings like 'yes', 'no', 'true', 'false'
        unique_vals = df[col].dropna().str.lower().unique()
        if set(unique_vals).issubset({'yes', 'no', 'true', 'false', '0', '1'}):
            df[col] = df[col].str.lower().map({'yes': True, 'no': False, 'true': True, 'false': False, '1': True, '0': False})

    return df

In [41]:
get_count_dtypes(df)

int           4
float        31
bool        173
object       35
datetime      0
other         0
dtype: int64


### convert True/False to 1/0

In [42]:
# Convert all boolean columns: True → 1, False → 0
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)


In [43]:
get_count_dtypes(df)

int         177
float        31
bool          0
object       35
datetime      0
other         0
dtype: int64


### Get object cols

In [44]:
# Get list of all object dtype columns
object_cols = df.select_dtypes(include='object').columns
print(object_cols)

Index(['City', 'ft', 'bt', 'transmission', 'owner', 'oem', 'model', 'price',
       'priceActual', 'trendingText.imgUrl', 'trendingText.heading',
       'trendingText.desc', 'Fuel Type', 'Seats', 'Kms Driven', 'RTO',
       'Ownership', 'Engine Displacement', 'Transmission', 'Mileage', 'Engine',
       'Max Power', 'Color', 'Engine Type', 'Max Torque',
       'Value Configuration', 'Fuel Suppy System', 'BoreX Stroke',
       'Turbo Charger', 'Super Charger', 'Drive Type', 'Steering Type',
       'Front Brake Type', 'Rear Brake Type', 'Tyre Type'],
      dtype='object')


In [45]:
df_object=df[object_cols]

In [46]:
df_object.head(2)

,City,ft,bt,transmission,owner,oem,model,price,priceActual,trendingText.imgUrl,...,Value Configuration,Fuel Suppy System,BoreX Stroke,Turbo Charger,Super Charger,Drive Type,Steering Type,Front Brake Type,Rear Brake Type,Tyre Type
0,Bangalore,Petrol,Hatchback,Manual,3rd Owner,Maruti,Maruti Celerio,₹ 4 Lakh,NaN,https://stimg.cardekho.com/used-cars/common/ic...,...,DOHC,MPFi,73 X 82 mm,No,False,FWD,Power,Ventilated Disc,Drum,"Tubeless, Radial"
1,Bangalore,Petrol,SUV,Manual,2nd Owner,Ford,Ford Ecosport,₹ 8.11 Lakh,NaN,https://stimg.cardekho.com/used-cars/common/ic...,...,DOHC,Direct Injection,79 X 76.5 mm,No,False,FWD,Power,Ventilated Disc,Drum,"Tubeless,Radial"


In [47]:
nunique_counts = df.select_dtypes(include='object').nunique()

# Filter columns with unique values > 10
cols_gt_10_unique = nunique_counts[nunique_counts > 10]
print(cols_gt_10_unique)

oem                      33
model                   315
price                  1400
priceActual             721
Kms Driven             4089
RTO                     464
Engine Displacement     150
Mileage                 522
Engine                  150
Max Power               760
Color                   153
Engine Type             508
Max Torque              617
Value Configuration      13
Fuel Suppy System        81
BoreX Stroke            171
Drive Type               20
Front Brake Type         34
Rear Brake Type          35
Tyre Type                34
dtype: int64


### select few imp columns and clean the data to float

In [48]:
df_tmp=df_object[['price','Kms Driven','Engine Displacement','Mileage','Max Power']]

In [49]:
# Clean price
def clean_price(val):
    if pd.isna(val):
        return None
    val = val.replace("₹", "").replace(",", "").strip().lower()
    if 'lakh' in val:
        return float(val.replace('lakh', '').strip()) * 1e5
    if 'crore' in val:
        return float(val.replace('crore', '').strip()) * 1e7
    return float(val)

# Apply transformations
df_tmp.loc[:, 'price'] = df_tmp['price'].apply(clean_price)


In [50]:
# Clean Kms Driven: remove commas and extract numeric values
df_tmp['Kms Driven'] = df_tmp['Kms Driven'].str.replace(',', '').str.extract('(\d+)').astype(float)

# Clean Engine Displacement: Extract digits from the "cc" part
df_tmp['Engine Displacement'] = df_tmp['Engine Displacement'].str.extract('(\d+)').astype(float)

# Clean Mileage: Extract the numeric value (assuming it's a float)
df_tmp['Mileage'] = df_tmp['Mileage'].str.extract(r'([\d.]+)').astype(float)

# Clean Max Power: Extract numeric part of the max power value
df_tmp['Max Power'] = df_tmp['Max Power'].str.replace(',', '').str.extract(r'([\d.]+)').astype(float)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
C:\Users\VINOTH\AppData\Local\Temp\ipykernel_17084\2004351802.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_tmp['Kms Driven'] = df_tmp['Kms Driven'].str.replace(',', '').str.extract('(\d+)').astype(float)
C:\Users\VINOTH\AppData\Local\Temp\ipykernel_17084\2004351802.py:5: SyntaxWarning: invalid escape sequence '\d'
  df_tmp['Engine Displacement'] = df_tmp['Engine Displacement'].str.extract('(\d+)').astype(float)
C:\Users\VINOTH\AppData\Local\Temp\ipykernel_17084\2004351802.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['Kms Drive

### merge the few cleaned columns into original df

In [51]:
df.update(df_tmp)

In [52]:
df=convert_object_to_int_float(df)

In [53]:
df['price']

0       400000.0
1       811000.0
2       585000.0
3       462000.0
4       790000.0
          ...   
8364    510000.0
8365    180000.0
8366    550000.0
8367    140000.0
8368    500000.0
Name: price, Length: 8369, dtype: float64

In [54]:
get_count_dtypes(df)

int         177
float        37
bool          0
object       29
datetime      0
other         0
dtype: int64


In [55]:
# Get list of all object dtype columns
object_cols = df.select_dtypes(include='object').columns
print(object_cols)
df_object =df[object_cols]

Index(['City', 'ft', 'bt', 'transmission', 'owner', 'oem', 'model',
       'priceActual', 'trendingText.imgUrl', 'trendingText.heading',
       'trendingText.desc', 'Fuel Type', 'Seats', 'RTO', 'Ownership',
       'Transmission', 'Engine', 'Color', 'Engine Type', 'Max Torque',
       'Value Configuration', 'Fuel Suppy System', 'BoreX Stroke',
       'Turbo Charger', 'Drive Type', 'Steering Type', 'Front Brake Type',
       'Rear Brake Type', 'Tyre Type'],
      dtype='object')


In [56]:
df.columns.tolist()

['City',
 'it',
 'ft',
 'bt',
 'km',
 'transmission',
 'ownerNo',
 'owner',
 'oem',
 'model',
 'modelYear',
 'centralVariantId',
 'variantName',
 'price',
 'priceActual',
 'priceSaving',
 'priceFixedText',
 'trendingText.imgUrl',
 'trendingText.heading',
 'trendingText.desc',
 'Registration Year',
 'Insurance Validity',
 'Fuel Type',
 'Seats',
 'Kms Driven',
 'RTO',
 'Ownership',
 'Engine Displacement',
 'Transmission',
 'Year of Manufacture',
 'Digital Odometer',
 'Digital Clock',
 'Centeral Locking',
 'Front Impact Beams',
 'Remote Fuel Lid Opener',
 'Rear Seat Headrest',
 'Accessory Power Outlet',
 'Wheel Covers',
 'Power Windows Rear',
 'Air Conditioner',
 'Fabric Upholstery',
 'Power Antenna',
 'Vanity Mirror',
 'Chrome Grille',
 'Rear Seat Belts',
 'Engine Immobilizer',
 'Side Impact Beams',
 'Remote Trunk Opener',
 'Door Ajar Warning',
 'Day Night Rear View Mirror',
 'Adjustable Head Lights',
 'Electronic Multi Tripmeter',
 'Halogen Headlamps',
 'Passenger Side Rear View Mirror'

In [57]:
selected_feature=['transmission','model','Year of Manufacture','City','Fuel Type','bt','Ownership','Kms Driven','Engine Type']
df_new=df[selected_feature]
print(df_new)

     transmission                  model  Year of Manufacture       City  \
0          Manual         Maruti Celerio               2015.0  Bangalore   
1          Manual          Ford Ecosport               2018.0  Bangalore   
2          Manual             Tata Tiago               2018.0  Bangalore   
3          Manual          Hyundai Xcent               2014.0  Bangalore   
4          Manual     Maruti SX4 S Cross               2015.0  Bangalore   
...           ...                    ...                  ...        ...   
8364       Manual         Maruti Celerio               2022.0    Kolkata   
8365       Manual        Maruti Alto 800               2014.0    Kolkata   
8366    Automatic  Mercedes-Benz C-Class               2011.0    Kolkata   
8367       Manual            Maruti Ritz               2012.0    Kolkata   
8368       Manual         Renault Duster               2017.0    Kolkata   

     Fuel Type         bt     Ownership  Kms Driven               Engine Type  
0      

In [58]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle



# Convert object columns to category
df_object = df_object.apply(lambda x: x.astype('category') if x.dtype == 'object' else x)

# Dictionary to store encoders
encoders = {}

# Fit and transform each column with LabelEncoder, and store the encoder
for col in df_object.select_dtypes(include='category').columns:
    le = LabelEncoder()
    df_object[col] = le.fit_transform(df_object[col])
    encoders[col] = le  # Save the encoder

# Save the encoders
with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(encoders, f)

print("Encoders saved!")



Encoders saved!


In [59]:
import pickle

# Load encoders
with open('label_encoders.pkl', 'rb') as f:
    encoders = pickle.load(f)

# Print encoding order
if encoders:
    for col, le in encoders.items():
        print(f"Column: {col}")
        for i, cls in enumerate(le.classes_):
            print(f"  {cls} → {i}")
        print("-" * 30)
else:
    print("❌ No encoders found. Make sure you saved them properly.")


Column: City
  Bangalore → 0
  Chennai → 1
  Delhi → 2
  Hyderabad → 3
  Jaipur → 4
  Kolkata → 5
------------------------------
Column: ft
  Cng → 0
  Diesel → 1
  Electric → 2
  Lpg → 3
  Petrol → 4
------------------------------
Column: bt
  Convertibles → 0
  Coupe → 1
  Hatchback → 2
  Hybrids → 3
  MUV → 4
  Minivans → 5
  Pickup Trucks → 6
  SUV → 7
  Sedan → 8
  Wagon → 9
  nan → 10
------------------------------
Column: transmission
  Automatic → 0
  Manual → 1
------------------------------
Column: owner
  0th Owner → 0
  1st Owner → 1
  2nd Owner → 2
  3rd Owner → 3
  4th Owner → 4
  5th Owner → 5
------------------------------
Column: oem
  Audi → 0
  BMW → 1
  Chevrolet → 2
  Citroen → 3
  Datsun → 4
  Fiat → 5
  Ford → 6
  Hindustan Motors → 7
  Honda → 8
  Hyundai → 9
  Isuzu → 10
  Jaguar → 11
  Jeep → 12
  Kia → 13
  Land Rover → 14
  Lexus → 15
  MG → 16
  Mahindra → 17
  Mahindra Renault → 18
  Mahindra Ssangyong → 19
  Maruti → 20
  Mercedes-Benz → 21
  Mini → 22
  

In [60]:
print(encoders)  # This should NOT be empty


{'City': LabelEncoder(), 'ft': LabelEncoder(), 'bt': LabelEncoder(), 'transmission': LabelEncoder(), 'owner': LabelEncoder(), 'oem': LabelEncoder(), 'model': LabelEncoder(), 'priceActual': LabelEncoder(), 'trendingText.imgUrl': LabelEncoder(), 'trendingText.heading': LabelEncoder(), 'trendingText.desc': LabelEncoder(), 'Fuel Type': LabelEncoder(), 'Seats': LabelEncoder(), 'RTO': LabelEncoder(), 'Ownership': LabelEncoder(), 'Transmission': LabelEncoder(), 'Engine': LabelEncoder(), 'Color': LabelEncoder(), 'Engine Type': LabelEncoder(), 'Max Torque': LabelEncoder(), 'Value Configuration': LabelEncoder(), 'Fuel Suppy System': LabelEncoder(), 'BoreX Stroke': LabelEncoder(), 'Turbo Charger': LabelEncoder(), 'Drive Type': LabelEncoder(), 'Steering Type': LabelEncoder(), 'Front Brake Type': LabelEncoder(), 'Rear Brake Type': LabelEncoder(), 'Tyre Type': LabelEncoder()}


In [61]:
df.update(df_object)

In [62]:
print(df.shape)

(8369, 243)


In [63]:
df=convert_object_to_int_float(df)

In [64]:
df['price']

0       400000.0
1       811000.0
2       585000.0
3       462000.0
4       790000.0
          ...   
8364    510000.0
8365    180000.0
8366    550000.0
8367    140000.0
8368    500000.0
Name: price, Length: 8369, dtype: float64

In [65]:
get_count_dtypes(df)

int         206
float        37
bool          0
object        0
datetime      0
other         0
dtype: int64


In [66]:
### All object columns are converted to numerical

In [67]:
df.to_csv('./cleaned_data/all_cars_cleaned_data.csv',index=False)

In [68]:
list(df.columns)
# or
#df.columns.tolist()




['City',
 'it',
 'ft',
 'bt',
 'km',
 'transmission',
 'ownerNo',
 'owner',
 'oem',
 'model',
 'modelYear',
 'centralVariantId',
 'variantName',
 'price',
 'priceActual',
 'priceSaving',
 'priceFixedText',
 'trendingText.imgUrl',
 'trendingText.heading',
 'trendingText.desc',
 'Registration Year',
 'Insurance Validity',
 'Fuel Type',
 'Seats',
 'Kms Driven',
 'RTO',
 'Ownership',
 'Engine Displacement',
 'Transmission',
 'Year of Manufacture',
 'Digital Odometer',
 'Digital Clock',
 'Centeral Locking',
 'Front Impact Beams',
 'Remote Fuel Lid Opener',
 'Rear Seat Headrest',
 'Accessory Power Outlet',
 'Wheel Covers',
 'Power Windows Rear',
 'Air Conditioner',
 'Fabric Upholstery',
 'Power Antenna',
 'Vanity Mirror',
 'Chrome Grille',
 'Rear Seat Belts',
 'Engine Immobilizer',
 'Side Impact Beams',
 'Remote Trunk Opener',
 'Door Ajar Warning',
 'Day Night Rear View Mirror',
 'Adjustable Head Lights',
 'Electronic Multi Tripmeter',
 'Halogen Headlamps',
 'Passenger Side Rear View Mirror'

In [69]:

[col for col in df.columns if 'Mileage' in col]


['Mileage']